In [ ]:
# derivative of y = x^2
def dy_dx(x):
  return 2*x

In [ ]:
dy_dx(4)

In [4]:
import torch

In [5]:
x = torch.tensor(3.0, requires_grad=True)

In [6]:
y = x**2

In [7]:
x

tensor(3., requires_grad=True)

In [8]:
y

tensor(9., grad_fn=<PowBackward0>)

In [9]:
y.backward()

In [10]:
x.grad

tensor(6.)

In [11]:
import math

"""
y = x^2
z = sin(y)

find dz/dx
"""

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [12]:
dz_dx(3)

-5.466781571308061

In [13]:
x = torch.tensor(3.0, requires_grad=True)

In [14]:
y = x**2

In [15]:
z = torch.sin(y)

In [16]:
x

tensor(3., requires_grad=True)

In [17]:
y

tensor(9., grad_fn=<PowBackward0>)

In [18]:
z

tensor(0.4121, grad_fn=<SinBackward0>)

In [19]:
# dz_dx
z.backward()

In [20]:
x.grad

tensor(-5.4668)

In [21]:
y.grad #would give an error.

<ipython-input-21-7afcbd95cdb6>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad #would give an error.


# A simple computation graph

In [22]:
import torch

x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0)
b = torch.tensor(0.0)

In [23]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [24]:
z = w * x + b
y_pred = torch.sigmoid(z)

loss = binary_cross_entropy_loss(y_pred, y)

In [25]:
loss

tensor(6.7012)

In [26]:
# Derivative calc.

dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

dy_pred_dz = y_pred * (1 - y_pred)

dz_dw = x
dz_db = 1

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [27]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


### using autograd

In [28]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [29]:
z = w * x + b
y_pred = torch.sigmoid(z)

loss = binary_cross_entropy_loss(y_pred, y)

In [30]:
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [31]:
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [32]:
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [33]:
loss.backward()

In [34]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


### lets now try with vector input tensor

In [37]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad = True)

In [39]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [40]:
y.backward()

In [42]:
# We get 3 gradients because we differentiate w.r.t x which is a vector. so it becomes gradient of y which means dely_delx1, dely_delx2, dely_delx3
x.grad

tensor([0.6667, 1.3333, 2.0000])

#### gradient clearing

when you run the backward function multiple times the gradient starts to accumulate, hence you'll need to clear it

In [44]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [45]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [46]:
y.backward()

In [47]:
x.grad

tensor(4.)

In [48]:
# but now if you run the forward pass and the backward pass again then the gradient syarts getting accumulated
y = x**2
y

y.backward()

In [49]:
x.grad

tensor(8.)

In [50]:
# once more
y = x**2
y

y.backward()

x.grad

tensor(12.)

In [51]:
# to stop this from happening you need to manually by doing
x.grad.zero_()
# the zero_() means its an inplace operation

y = x**2
y

y.backward()

x.grad

tensor(4.)

### Disabling gradient tracking

- Say you have a neural network being trained and you're calculating the derivatives required in training. So you'll keep gradient tracking on as you need the gradients for backpropagation

- But once the training is done, you don't need to recalculate the gradients, you just need to infer using the weights obtained, so then you'll switch off gradient tracking.

In [58]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [59]:
# forward pass lets say

y = x**2
y

tensor(9., grad_fn=<PowBackward0>)

In [60]:
# backward pass say
y.backward()

# so then the obtained gradient
x.grad

tensor(6.)

In [61]:
x

tensor(3., requires_grad=True)

In [62]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

#### Option 1 - requires_grad_(False)

In [63]:
x.requires_grad_(False)

tensor(3.)

In [64]:
x

tensor(3.)

In [65]:
y = x ** 2

In [66]:
y

tensor(9.)

In [68]:
# because requires_grad is switched off so this doesn't work
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

#### Option 2 - detach

In [69]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [71]:
# this creates a new tensor itself
z = x.detach()
z

tensor(2.)

In [72]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [74]:
y1 = z ** 2
y1

tensor(4.)

In [75]:
y.backward() #works

In [76]:
y1.backward() #wont work

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

#### Option 3 - torch.no_grad()

- this is the most convenient way

In [77]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [79]:
# make use of the with statement

with torch.no_grad():
    y = x ** 2

y

tensor(4.)

In [80]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [81]:
# if you remove the with statment then


y = x ** 2

y

tensor(4., grad_fn=<PowBackward0>)

In [82]:
y.backward()